In [11]:
import random
import time
import numpy as np
from tabulate import tabulate
import copy

class Process:
    def __init__(self, pid, burst_time, arrival_time):
        self.pid = pid
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.arrival_time = arrival_time
        self.turnaround_time = 0
        self.waiting_time = 0
        self.response_time = -1
        self.start_time = 0


def first_come_first_serve(processes):
    processes.sort(key=lambda p: p.arrival_time)
    
    time_elapsed = 0
    for process in processes:
        if time_elapsed < process.arrival_time:
            time_elapsed = process.arrival_time
        
        process.start_time = time_elapsed
        process.waiting_time = time_elapsed - process.arrival_time
        process.response_time = process.start_time - process.arrival_time
        time_elapsed += process.burst_time
        process.turnaround_time = time_elapsed - process.arrival_time
        process.remaining_time = 0
        
def round_robin(processes, time_quantum):
    remaining = len(processes) 
    processes.sort(key=lambda p: p.arrival_time) 
    time = processes[0].arrival_time  
    
    while remaining > 0:
        noprocready = True  
        
        for process in processes:
            if process.arrival_time <= time and process.remaining_time > 0:
                noprocready = False 
                
                if process.start_time == 0:
                    process.start_time = time
                    process.response_time = time - process.arrival_time
                
                if process.remaining_time > time_quantum:
                    time += time_quantum
                    process.remaining_time -= time_quantum
                else: 
                    time += process.remaining_time
                    process.waiting_time = time - process.burst_time - process.arrival_time
                    process.turnaround_time = time - process.arrival_time
                    process.remaining_time = 0
                    remaining -= 1  
        
        if noprocready:
            time += 1

def shortest_remaining_time_first(processes):
    
    processes.sort(key=lambda p: p.arrival_time)
    time = processes[0].arrival_time

    while any(p.remaining_time > 0 for p in processes):  
        available_processes = [p for p in processes if p.arrival_time <= time and p.remaining_time > 0]
        
        if available_processes:
            current_process = min(available_processes, key=lambda p: p.remaining_time)
            
            if current_process.start_time == 0:
                current_process.start_time = time
                current_process.response_time = time - current_process.arrival_time
            
            current_process.remaining_time -= 1
            time += 1

            if current_process.remaining_time == 0:
                current_process.turnaround_time = time - current_process.arrival_time
                current_process.waiting_time = current_process.turnaround_time - current_process.burst_time
        else:
            time += 1

def display_schedule(processes):
    headers = ["Process ID", "Burst", "Remaining", "Arrival", "Start", "Turnaround", "Waiting", "Response"]
    table = [
        [
            process.pid,
            process.burst_time,
            process.remaining_time,
            process.arrival_time,
            process.start_time,
            process.turnaround_time,
            process.waiting_time,
            process.response_time,
        ]
        for process in processes
    ]
    print(tabulate(table, headers=headers, tablefmt="pretty"))


def main():
    n = int(input("Enter the number of processes: "))
    time_quantum = int(input("Enter the time quantum for Round Robin: "))
    random.seed(int(time.time()))  
    
    arrival_min = 0
    arrival_max = 50

    burst_mean = 10
    burst_stddev = 3

    processes = []
    for i in range(n):
        arrival_time = random.randint(arrival_min, arrival_max)  
        burst_time = max(1, int(np.random.normal(burst_mean, burst_stddev))) 
        processes.append(Process(i + 1, burst_time, arrival_time))
    processes_copy = copy.deepcopy(processes)
    first_come_first_serve(processes_copy)
    print('First Come First Serve: ')
    display_schedule(processes_copy)
    print("\n")
    processes_copy = copy.deepcopy(processes)
    print('Round Robin: ')
    round_robin(processes_copy, time_quantum)
    display_schedule(processes_copy)
    print("\n")
    processes_copy = copy.deepcopy(processes)
    print('Shortest Remaining Time First: ')
    shortest_remaining_time_first(processes_copy)
    display_schedule(processes_copy)
    
    
if __name__ == "__main__":
    main()


First Come First Serve: 
+------------+-------+-----------+---------+-------+------------+---------+----------+
| Process ID | Burst | Remaining | Arrival | Start | Turnaround | Waiting | Response |
+------------+-------+-----------+---------+-------+------------+---------+----------+
|     3      |   8   |     0     |    3    |   3   |     8      |    0    |    0     |
|     1      |   8   |     0     |   11    |  11   |     8      |    0    |    0     |
|     4      |   7   |     0     |   11    |  19   |     15     |    8    |    8     |
|     2      |  10   |     0     |   35    |  35   |     10     |    0    |    0     |
+------------+-------+-----------+---------+-------+------------+---------+----------+


Round Robin: 
+------------+-------+-----------+---------+-------+------------+---------+----------+
| Process ID | Burst | Remaining | Arrival | Start | Turnaround | Waiting | Response |
+------------+-------+-----------+---------+-------+------------+---------+----------+
| 